In [5]:
import numpy as np
from scipy.stats import t as student_t

In [6]:
def get_m_v(Y, C_pre, Psi_pre_inv, X1, X2, tau, sample = 'sample'):

    n = Y.shape[0]
    
    v = np.zeros(n)
    m = np.zeros(n)


    gamma = np.zeros(n) # point estimate of gamma

    if X1 is not None:
        r1 = X1.shape[1]
        A = C_pre[:,:r1]
        B = C_pre[:,r1:]
    else:
        B = C_pre


    v = np.abs(1+(tau[:,np.newaxis]*X2*np.dot(X2, np.dot(B.T, Psi_pre_inv).dot(B))).sum(1)) ** (-1)
    if X1 is not None:
        errors = Y - np.dot(X1,A.T)
    else:
        errors = Y.copy()

    m =  v*(errors*np.dot(tau[:,np.newaxis]*X2, B.T).dot(Psi_pre_inv)).sum(1)

    if sample == 'sample':
            gamma = np.random.normal(loc = m, scale = v**0.5)
    else:
        gamma = m


    return m, v, gamma


def get_tau(Y, X1,X2, gamma, C_pre, Psi_est_inv, nu, sample = 'sample'):

    n = Y.shape[0]
    d = Y.shape[1]
    
    tau_mean = np.zeros(n)  # mean for M
    tau = np.zeros(n)  # point estimate to be used in get v_e

    if X1 is not None:
        r1 = X1.shape[1]
        A = C_pre[:,:r1]
        B = C_pre[:,r1:]
    else:
        B = C_pre

    alpha_1 = float((nu+d)/2.0)
    if X1 is not None:
        tmp = Y - np.dot(X1, A.T) - gamma[:, np.newaxis]*np.dot(X2, B.T)
    else:
        tmp = Y - gamma[:, np.newaxis]*np.dot(X2, B.T)

    beta = (nu + (tmp * np.dot(tmp, Psi_est_inv)).sum(1))/2.0
    if sample == 'sample':
        tau = np.random.gamma(shape = alpha_1, scale = 1/beta)
    elif sample == 'mode':
            tau = (alpha_1-1)/beta
    tau_mean = alpha_1/beta


    return tau_mean, tau



def solve_B_psi( X_tilde, Y_tilde, n):


    # M-step
    try:
        C = np.dot(Y_tilde.T, X_tilde).dot(np.linalg.inv(np.dot(X_tilde.T,X_tilde)))
    except:
        C = np.dot(Y_tilde.T, X_tilde).dot(np.linalg.pinv(np.dot(X_tilde.T,X_tilde)))

    Psi_est = np.dot((Y_tilde-np.dot(X_tilde,C.T)).T, (Y_tilde-np.dot(X_tilde,C.T)))/n
    try:
        Psi_est_inv = np.linalg.inv(Psi_est)
    except:
        Psi_est_inv = np.linalg.pinv(Psi_est)


    return C, Psi_est, Psi_est_inv



def get_X_tilde( X1, X2, m,v, tau):
        
        n = X2.shape[0]




        tau_mix = np.sqrt(tau[:,np.newaxis])

        s = v[:,np.newaxis] ** 0.5

        if X1 is not None:
            r1 = X1.shape[1]
            X_tilde =  np.vstack((np.hstack((X1*tau_mix, tau_mix*m[:,np.newaxis]*X2)),np.hstack((np.zeros((n, r1)),tau_mix*s*X2))))
        else:
            X_tilde =  np.vstack((tau_mix*m[:,np.newaxis]*X2,tau_mix*s*X2))

        
        return X_tilde
    

def get_X_bar( X1, X2, tau, gamma):

    if X1 is None:
        with_mean = False
        tau_mix = np.sqrt(tau[:,np.newaxis])
    
    gamma = gamma[:,np.newaxis]


    if with_mean:
        X_bar =  np.hstack((X1*tau_mix, tau_mix*gamma*X2))
    elif not with_mean:
        X_bar =  tau_mix*gamma*X2

    
    return X_bar
    
def get_Y_bar(Y, tau):
    return np.sqrt(tau[:, np.newaxis])*Y
    

def get_Y_tilde( Y, tau):

    n = Y.shape[0]
    d = Y.shape[1]
    

    tau_mix = np.sqrt(tau[:,np.newaxis])

    return  np.vstack((Y*tau_mix, np.zeros((n,d))))

def one_step(Y, X1, X2, A, B, Psi_inv, tau, gamma, nu, sample):
    n = Y.shape[0]


    if A is not None:
        C = np.hstack((A,B))
    else:
        C = B

    m,v, gamma = get_m_v(Y, C, Psi_inv, X1, X2, tau, sample = sample)

    X_tilde = get_X_tilde(X1, X2, m,v, tau)
    Y_tilde = get_Y_tilde(Y, tau)

    C_est, Psi_est, Psi_pre_inv = solve_B_psi(X_tilde, Y_tilde, n)

    #if error is t we have to do another EM step
    tau_mean, tau = get_tau(X1, X2, gamma, C_est, Psi_pre_inv, nu, sample)
    X_bar =  get_X_bar(X1, X2, tau_mean, gamma)
    Y_bar =  get_Y_bar(tau_mean)

    C_est, Psi_est, Psi_pre_inv = solve_B_psi(X_bar, Y_bar, n)

    return C_est, Psi_est, Psi_pre_inv, tau_mean, tau, m,v,gamma


def one_step_wrong(Y, X1, X2, A, B, Psi_inv, tau, gamma, nu, sample):
    n = Y.shape[0]


    if A is not None:
        C = np.hstack((A,B))
    else:
        C = B


    m,v, gamma = get_m_v(Y, C, Psi_inv, X1, X2, tau, sample = 'mode')
    m = m
    v = v
    gamma = gamma


    tau_mean, tau = get_tau(X1, X2, gamma, C_est, Psi_pre_inv, nu, sample = 'mode')


    X_tilde = get_X_tilde(X1, X2, m,v, tau_mean)
    Y_tilde = get_Y_tilde(Y, tau_mean)
    C_est, Psi_est, Psi_pre_inv = solve_B_psi(X_tilde, Y_tilde,n)

    return C_est, Psi_est, Psi_pre_inv, tau_mean, tau, m,v,gamma





Simulate data

In [12]:

tol = 1e-6
max_iter = 200
nu = 5
rnd = np.random.RandomState(42)

n = 1000
r = 5
d = 3


        


X = rnd.normal(loc = 0, scale = 1, size = (n,r))


# Generate coef matrix
B_true = rnd.normal(loc = 0, scale = 1, size = (d,r))
B_true[np.abs(B_true)<0.7] = 0


# Generate observations

st_dist = student_t(df = nu, scale = 1)
st_dist.random_state = np.random.RandomState(5)


gamma = rnd.normal(loc = 0, scale = 1, size = (n))
epsilon = st_dist.rvs(size = (n, d))

Y_cov = gamma[:, np.newaxis]*np.dot(X, B_true.T) + epsilon





In [13]:

B_init = B_true# np.ones((d,r))
psi_init =np.identity(d)*(nu-1)/nu
Psi_inv_init = np.linalg.inv(psi_init)
tau_init = np.ones(n)
gamma_init = np.ones(n)


#Y, X1, X2, A, B, Psi_inv, tau, gamma, nu, sample

C_1, Psi_1, Psi_inv_1, tau_mean_1, tau_1, m_1,v_1,gamma_1 = one_step(Y_cov, None, X, None, B_init, Psi_inv_init, tau_init, gamma, nu, 'mode')
C_2, Psi_2, Psi_inv_2, tau_mean_2, tau_2, m_2,v_2,gamma_2 = one_step_wrong(Y_cov, None, X, None, B_init, Psi_inv_init, tau_init, gamma, nu, None)


TypeError: one_step() takes 9 positional arguments but 10 were given